In [21]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils

img = cv2.imread('t011.tif')

top_filter = (13,13)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  top_filter) 
#blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
#blackhat = cv2.cvtColor(blackhat, cv2.COLOR_BGR2GRAY)
tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
tophat = cv2.cvtColor(tophat, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(tophat,(35,35),0)
ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

inverse_mask = cv2.bitwise_not(thresh)

blur = cv2.medianBlur(img,25)
denoise = cv2.fastNlMeansDenoising(blur, None, 7, 21, 20)
#kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (25,25))
#tophat = cv2.morphologyEx(denoise, cv2.MORPH_TOPHAT, kernel)
denoise = cv2.cvtColor(denoise, cv2.COLOR_BGR2GRAY)
equalized = cv2.equalizeHist(denoise)
#equalized = cv2.bitwise_not(equalized)

ret,thresh = cv2.threshold(equalized,180,255,cv2.THRESH_BINARY)

lessened = cv2.subtract(thresh, inverse_mask)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (5,5)) 
lessened = cv2.erode(lessened,kernel,iterations=1)
 
ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create()
ridges = ridge_filter.getRidgeFilteredImage(lessened) 

#final = cv2.subtract(ridges, inverse_mask)

contours = cv2.findContours(lessened, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
#cv2.drawContours(img, contours, -1, (0,255,0), 3)

areas = []
cells = []

for cont in contours:
    area = cv2.contourArea(cont)
    areas.append(int(area))
    if area > 1000:
        cells.append(cont)
cv2.drawContours(img, cells, -1, (0,255,0), 3)
#areas.sort()
#print(areas)

cv2.imshow("img", img)
cv2.imshow("equalized", equalized)
cv2.imshow("blur", blur)
cv2.imshow("denoise", denoise)
cv2.imshow("thresh", thresh)
cv2.imshow("ridges", ridges)
cv2.imshow("final", lessened)
cv2.imshow("inverse", inverse_mask)
#cv2.imshow("pathways", pathway_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()